In [2]:
import torch
import torch.nn as nn
import utility_functions as uf
import pandas as pd
import numpy as np
import importlib

In [3]:
pd.options.display.max_columns = None
importlib.reload(uf)

<module 'utility_functions' from 'c:\\Users\\emilw\\Assignment3\\sepsis-ts\\SepsisTS\\utility_functions.py'>

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Data Loading

In [23]:
# Loading the data from helper function
timeseries = uf.TimeseriesTorch('raw_data/', 43)

In [28]:
print(timeseries.train[0][0])
print(timeseries.train[0][1])
print(timeseries.train[1])

tensor([-0.5305, -0.6444, -0.7069, -0.1681, -0.2753, -1.3732, -0.0414, -0.2701,
        -0.4715, -0.1961, -0.2391, -0.3323, -0.3006, -0.0052, -0.3205, -0.3258,
         0.0641, -0.5208, -1.2396, -0.2687, -0.6543, -1.0946,  1.8565, -0.8675,
        -0.8116, -0.2486, -0.3816, -0.3783, -1.9098,  0.8032, -0.9145, -1.1529,
        -0.5172,  1.0191, -0.3347,  0.0107, -0.5110, -0.3339, -0.2847, -0.3229,
         0.3171,  0.0617, -0.2662])
tensor([-0.5305, -0.6444, -0.7069, -0.1681, -0.2753, -1.3732, -0.0414, -0.2701,
        -0.4715, -0.1961, -0.2391, -0.3323, -0.3006, -0.0052, -0.3205, -0.3258,
         0.0641, -0.5208, -1.2396, -0.2687, -0.6543, -1.0946,  1.8565, -0.8675,
        -0.8116, -0.2486, -0.3816, -0.3783, -1.9098,  0.8032, -0.9145, -1.1529,
        -0.5172,  1.0191, -0.3347,  0.0107, -0.5110, -0.3339, -0.2847, -0.3229,
         0.3171,  0.0617, -0.2662])
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [6]:
# Loading the data manually
data1 = pd.read_table("raw_data/sepsisexp_timeseries_partition-A.tsv")
data2 = pd.read_table("raw_data/sepsisexp_timeseries_partition-B.tsv")
data3 = pd.read_table("raw_data/sepsisexp_timeseries_partition-C.tsv")
data4 = pd.read_table("raw_data/sepsisexp_timeseries_partition-D.tsv")

In [7]:
# Setting partitions. 
# The dataset is split into four partitions (A–D) that are used in the 4-fold cross validation experiments in (Schamoni et al., 2022). 
data1["partition"] = 1
data2["partition"] = 2
data3["partition"] = 3
data4["partition"] = 4

In [8]:
data_orig = pd.concat([data1, data2, data3, data4], ignore_index=True)

In [56]:
df = data_orig.copy()

In [12]:
df.isna().sum()

id                                0
sepsis                            0
severity                          0
timestep                          0
respiratory_minute_volume         0
heart_rate                        0
leukocytes                        0
temperature                       0
partial_co2                       0
respiratory_rate                  0
arterial_ph                       0
bilirubin                         0
blood_urea_nitrogen               0
creatinine                        0
diastolic_bp                      0
fraction_of_inspired_o2           0
mean_bp                           0
partial_pressure_art._o2          0
systolic_bp                       0
thrombocytes                      0
horowitz_index                    0
bun/creatinine_ratio              0
delta-temperature                 0
lactate                           0
bicarbonate                       0
c-reactive_protein                0
hemoglobin                        0
heart_time_volume           

In [124]:
df.shape

(602568, 52)

In [10]:
df["timestep"]


0           0.0
1           0.5
2           1.0
3           1.5
4           2.0
          ...  
602563    190.5
602564    191.0
602565    191.5
602566    192.0
602567    192.5
Name: timestep, Length: 602568, dtype: float64

In [16]:

for partition, group_data in df.groupby('partition'):
    print(f"Partition: {partition}")
    print(group_data.head(), "\n")



Partition: 1
      id  sepsis  severity  timestep  respiratory_minute_volume  heart_rate  \
0  12292       0       0.0       0.0                   0.190898    0.424464   
1  12292       0       0.0       0.5                   0.157654    0.667394   
2  12292       0       0.0       1.0                   0.024678    0.618808   
3  12292       0       0.0       1.5                  -0.208030    0.278706   
4  12292       0       0.0       2.0                  -0.108298   -0.352912   

   leukocytes  temperature  partial_co2  respiratory_rate  arterial_ph  \
0    0.301015    -0.168117    -0.275272          1.879692    -0.041447   
1    0.301015    -0.168117    -0.275272          1.708485    -0.041447   
2    0.301015    -0.732387     1.003408          2.050899     1.804109   
3    0.301015    -0.732387     1.003408          1.366071     1.804109   
4    0.301015    -0.732387     1.094023          1.537278     1.677315   

   bilirubin  blood_urea_nitrogen  creatinine  diastolic_bp  \
0   

In [118]:
#Removes rows for each patient after severity>=2. 
#Also marks the last (input hour) rows as targets.

def setSepsisColumn(df_input, colName, hours):
    hours = hours * 2
    df_input[colName] = 0
    df_result = df_input[df_input["severity"] < 2]

    for pid in df_result["id"].unique():
        last_n_per_id = df_result[df_result["id"]==pid].tail(hours).index
        #print(last_n_per_id)
        df_result.loc[last_n_per_id, colName] = 1

    return df_result



df_test = df.copy()
df_test = setSepsisColumn(df_test, "2hourSepsis", 2)
df_test = setSepsisColumn(df_test, "4hourSepsis", 4)
df_test = setSepsisColumn(df_test, "6hourSepsis", 6)

           id  sepsis  severity  timestep  respiratory_minute_volume  \
126     12292       0       0.0      63.0                  -3.033777   
127     12292       0       0.0      63.5                  -3.033777   
128     12292       0       0.0      64.0                  -3.033777   
129     12292       0       0.0      64.5                  -3.033777   
130     12292       0       0.0      65.0                  -3.033777   
...       ...     ...       ...       ...                        ...   
602563  13304       1       0.0     190.5                   1.022000   
602564  13304       1       0.0     191.0                   1.022000   
602565  13304       1       0.0     191.5                   1.022000   
602566  13304       1       0.0     192.0                   1.022000   
602567  13304       1       0.0     192.5                   1.022000   

        heart_rate  leukocytes  temperature  partial_co2  respiratory_rate  \
126       0.473050   -0.202947    -1.183803     0.711426 

df_test[df_test["2hourSepsis"]==1].count()

In [139]:
unique_id=df_test['id'].unique
unique_ids_count_one_2h = df_test[df_test["2hourSepsis"] == 1]['id'].nunique()
unique_ids_count_zero_2h = df_test[df_test["2hourSepsis"] == 0]['id'].nunique()

print("2h 1:", unique_ids_count_one_2h)
print("2h 0:", unique_ids_count_zero_2h)

print(df_test["2hourSepsis"].value_counts())
print(df_test["2hourSepsis"].value_counts())
print(df_test["4hourSepsis"].value_counts())
print(df_test["6hourSepsis"].value_counts())
print(df_test["sepsis"].value_counts())
print(df["sepsis"].value_counts())


2h 1: 1275
2h 0: 1275
2hourSepsis
0    421742
1      5100
Name: count, dtype: int64
2hourSepsis
0    421742
1      5100
Name: count, dtype: int64
4hourSepsis
0    416642
1     10200
Name: count, dtype: int64
6hourSepsis
0    411542
1     15300
Name: count, dtype: int64
sepsis
0    237823
1    189019
Name: count, dtype: int64
sepsis
1    364745
0    237823
Name: count, dtype: int64


df_sort = df.sort_values(by=['id','timestep'])
df_sort_sepsis = df_sort[df_sort["sepsis"] == 1]

for pid in df_sort_sepsis['id'].unique():
    id_index = df_sort_sepsis[df_sort_sepsis["id"] == pid].index
    for i in range(len(id_index)):
        if i + 8 < len(id_index):
            if 1 in df.loc[id_index[i:i+8], 'severity'].values and 2 in df.loc[id_index[i+1:i+9], 'severity'].values:
                df_sort_sepsis.at[id_index[i], '4hourSepsis'] = 1
                # Drop rows where id_index > i for the same pid
                #df_sort_sepsis.drop(id_index[i+10:], inplace=True)



df_sort=df.copy()

df_sort['FirstOccurrenceFlag'] = 0

for pid in df_sort['id'].unique():
    # Extract indices for this patient where sepsis == 1
    patient_indices = df_sort[(df_sort['id'] == pid) & (df_sort['sepsis'] == 1)].index
    
    # Check the condition and flag the first occurrence
    for i in range(len(patient_indices)):
        if i + 8 < len(patient_indices):
            current_indices = patient_indices[i:i+8]
            next_indices = patient_indices[i+1:i+9]
            if 1 in df_sort.loc[current_indices, 'severity'].values and 2 in df_sort.loc[next_indices, 'severity'].values:
                df_sort.at[current_indices[0], 'FirstOccurrenceFlag'] = 1
                break  # Stop after flagging the first occurrence



## Preprocess

In [ ]:
df_prep = df_test.copy()

In [153]:
cols_to_drop = ["id", "sepsis", "severity"]
df_test.drop(columns=cols_to_drop)

,timestep,respiratory_minute_volume,heart_rate,leukocytes,temperature,partial_co2,respiratory_rate,arterial_ph,bilirubin,blood_urea_nitrogen,creatinine,diastolic_bp,fraction_of_inspired_o2,mean_bp,partial_pressure_art._o2,systolic_bp,thrombocytes,horowitz_index,bun/creatinine_ratio,delta-temperature,lactate,bicarbonate,c-reactive_protein,hemoglobin,heart_time_volume,lymphocytes,sodium,pancreatic_lipase,procalcitonin,quick_score,oxygen_saturation,blood_glucose,base_excess,chloride,calcium,potassium,mixed_venous_oxygen_saturation,urine_output,net balance,alanine_transaminase,aspartate_transaminase,stroke_volume,svri,age,partition,2hourSepsis,4hourSepsis,6hourSepsis,time_left
0,0.0,0.190898,0.424464,0.301015,-0.168117,-0.275272,1.879692,-0.041447,0.179544,1.445381,2.396762,-0.239052,1.479290,-0.300612,-0.005231,-0.320473,1.114980,-0.938897,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
1,0.5,0.157654,0.667394,0.301015,-0.168117,-0.275272,1.708485,-0.041447,0.179544,1.445381,2.396762,-0.936105,0.573504,-1.329784,-0.005231,-1.485067,1.114980,-0.562761,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
2,1.0,0.024678,0.618808,0.301015,-0.732387,1.003408,2.050899,1.804109,0.179544,1.445381,2.396762,-1.424042,1.026397,-1.615666,-1.310985,-1.368608,1.114980,-1.498923,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
3,1.5,-0.208030,0.278706,0.301015,-0.732387,1.003408,1.366071,1.804109,0.179544,1.445381,2.396762,-1.284632,6.008223,-1.444137,-1.310985,-1.368608,1.114980,-2.018409,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
4,2.0,-0.108298,-0.352912,0.301015,-0.732387,1.094023,1.537278,1.677315,0.179544,1.445381,2.396762,-1.284632,1.116975,-1.386961,-0.977793,-1.368608,1.114980,-1.337929,-0.740976,-0.377821,-0.268699,4.092377,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-1.733639,0.255646,5.165209,-3.128827,-0.868157,2.719423,0.010733,-0.710447,0.452120,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602563,190.5,1.022000,-0.255740,0.500221,0.057592,0.459717,1.023658,0.423464,-0.451520,-0.063366,-0.629891,-0.308758,-1.056912,0.499856,-1.587144,1.154679,1.242303,-0.826056,0.703425,-0.894861,-0.463905,1.266985,0.468437,0.454265,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.475706,1.033018,-0.971420,0.621769,0.683373,0.753742,0.087333,-0.429313,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,0,1,1,6
602564,191.0,1.022000,-0.401498,0.500221,0.057592,-0.275272,1.023658,0.874287,-0.451520,-0.063366,-0.629891,-0.657284,-1.056912,0.328327,0.054803,1.115859,1.242303,1.044368,0.703425,-0.894861,-0.561509,0.634098,0.468437,0.135578,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.573219,0.344320,-0.630777,0.224456,0.072557,0.753742,-0.710447,-0.413381,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,1,1,1,6
602565,191.5,1.022000,-0.255740,0.500221,-0.055263,-0.275272,1.023658,0.874287,-0.451520,-0.063366,-0.629891,-0.099642,-1.056912,0.614

In [152]:
df_2hr = df_test.drop(["4hourSepsis", "6hourSepsis", ], axis=1)
df_2hr

df_4hr = df_test.drop(["2hourSepsis", "6hourSepsis", ], axis=1)
df_4hr

df_6hr = df_test.drop(["2hourSepsis", "4hourSepsis", ], axis=1)
df_6hr

,id,sepsis,severity,timestep,respiratory_minute_volume,heart_rate,leukocytes,temperature,partial_co2,respiratory_rate,arterial_ph,bilirubin,blood_urea_nitrogen,creatinine,diastolic_bp,fraction_of_inspired_o2,mean_bp,partial_pressure_art._o2,systolic_bp,thrombocytes,horowitz_index,bun/creatinine_ratio,delta-temperature,lactate,bicarbonate,c-reactive_protein,hemoglobin,heart_time_volume,lymphocytes,sodium,pancreatic_lipase,procalcitonin,quick_score,oxygen_saturation,blood_glucose,base_excess,chloride,calcium,potassium,mixed_venous_oxygen_saturation,urine_output,net balance,alanine_transaminase,aspartate_transaminase,stroke_volume,svri,age,partition,6hourSepsis,time_left
0,12292,0,0.0,0.0,0.190898,0.424464,0.301015,-0.168117,-0.275272,1.879692,-0.041447,0.179544,1.445381,2.396762,-0.239052,1.479290,-0.300612,-0.005231,-0.320473,1.114980,-0.938897,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,6
1,12292,0,0.0,0.5,0.157654,0.667394,0.301015,-0.168117,-0.275272,1.708485,-0.041447,0.179544,1.445381,2.396762,-0.936105,0.573504,-1.329784,-0.005231,-1.485067,1.114980,-0.562761,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,6
2,12292,0,0.0,1.0,0.024678,0.618808,0.301015,-0.732387,1.003408,2.050899,1.804109,0.179544,1.445381,2.396762,-1.424042,1.026397,-1.615666,-1.310985,-1.368608,1.114980,-1.498923,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,6
3,12292,0,0.0,1.5,-0.208030,0.278706,0.301015,-0.732387,1.003408,1.366071,1.804109,0.179544,1.445381,2.396762,-1.284632,6.008223,-1.444137,-1.310985,-1.368608,1.114980,-2.018409,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,6
4,12292,0,0.0,2.0,-0.108298,-0.352912,0.301015,-0.732387,1.094023,1.537278,1.677315,0.179544,1.445381,2.396762,-1.284632,1.116975,-1.386961,-0.977793,-1.368608,1.114980,-1.337929,-0.740976,-0.377821,-0.268699,4.092377,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-1.733639,0.255646,5.165209,-3.128827,-0.868157,2.719423,0.010733,-0.710447,0.452120,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602563,13304,1,0.0,190.5,1.022000,-0.255740,0.500221,0.057592,0.459717,1.023658,0.423464,-0.451520,-0.063366,-0.629891,-0.308758,-1.056912,0.499856,-1.587144,1.154679,1.242303,-0.826056,0.703425,-0.894861,-0.463905,1.266985,0.468437,0.454265,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.475706,1.033018,-0.971420,0.621769,0.683373,0.753742,0.087333,-0.429313,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,1,6
602564,13304,1,0.0,191.0,1.022000,-0.401498,0.500221,0.057592,-0.275272,1.023658,0.874287,-0.451520,-0.063366,-0.629891,-0.657284,-1.056912,0.328327,0.054803,1.115859,1.242303,1.044368,0.703425,-0.894861,-0.561509,0.634098,0.468437,0.135578,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.573219,0.344320,-0.630777,0.224456,0.072557,0.753742,-0.710447,-0.413381,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,1,6
602565,13304,1,0.0,191.5,1.022000,-0.255740,0.500221,-0.055263,-0.275272,1.0236

In [ ]:
df_2hr_ts = uf.TimeseriesTorch(df_2hr)

,id,sepsis,severity,timestep,respiratory_minute_volume,heart_rate,leukocytes,temperature,partial_co2,respiratory_rate,arterial_ph,bilirubin,blood_urea_nitrogen,creatinine,diastolic_bp,fraction_of_inspired_o2,mean_bp,partial_pressure_art._o2,systolic_bp,thrombocytes,horowitz_index,bun/creatinine_ratio,delta-temperature,lactate,bicarbonate,c-reactive_protein,hemoglobin,heart_time_volume,lymphocytes,sodium,pancreatic_lipase,procalcitonin,quick_score,oxygen_saturation,blood_glucose,base_excess,chloride,calcium,potassium,mixed_venous_oxygen_saturation,urine_output,net balance,alanine_transaminase,aspartate_transaminase,stroke_volume,svri,age,partition,2hourSepsis,4hourSepsis,6hourSepsis,time_left
0,12292,0,0.0,0.0,0.190898,0.424464,0.301015,-0.168117,-0.275272,1.879692,-0.041447,0.179544,1.445381,2.396762,-0.239052,1.479290,-0.300612,-0.005231,-0.320473,1.114980,-0.938897,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
1,12292,0,0.0,0.5,0.157654,0.667394,0.301015,-0.168117,-0.275272,1.708485,-0.041447,0.179544,1.445381,2.396762,-0.936105,0.573504,-1.329784,-0.005231,-1.485067,1.114980,-0.562761,-0.740976,-1.239555,-0.268699,-0.654281,0.416335,1.856485,-0.867504,-0.811566,-0.248615,0.781767,-0.349756,-0.184841,-0.782330,-0.914517,-1.152851,-0.517229,1.019083,-0.334653,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
2,12292,0,0.0,1.0,0.024678,0.618808,0.301015,-0.732387,1.003408,2.050899,1.804109,0.179544,1.445381,2.396762,-1.424042,1.026397,-1.615666,-1.310985,-1.368608,1.114980,-1.498923,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
3,12292,0,0.0,1.5,-0.208030,0.278706,0.301015,-0.732387,1.003408,1.366071,1.804109,0.179544,1.445381,2.396762,-1.284632,6.008223,-1.444137,-1.310985,-1.368608,1.114980,-2.018409,-0.740976,-0.377821,-0.171096,4.114980,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-2.050742,0.401916,5.255039,-3.242374,-0.868157,2.923028,0.010733,-0.710447,1.259337,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
4,12292,0,0.0,2.0,-0.108298,-0.352912,0.301015,-0.732387,1.094023,1.537278,1.677315,0.179544,1.445381,2.396762,-1.284632,1.116975,-1.386961,-0.977793,-1.368608,1.114980,-1.337929,-0.740976,-0.377821,-0.268699,4.092377,0.416335,-0.884218,-0.867504,-0.811566,-0.010212,0.781767,-0.349756,-0.184841,-1.733639,0.255646,5.165209,-3.128827,-0.868157,2.719423,0.010733,-0.710447,0.452120,-0.023852,0.117472,0.317126,0.061715,0.371047,1,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602563,13304,1,0.0,190.5,1.022000,-0.255740,0.500221,0.057592,0.459717,1.023658,0.423464,-0.451520,-0.063366,-0.629891,-0.308758,-1.056912,0.499856,-1.587144,1.154679,1.242303,-0.826056,0.703425,-0.894861,-0.463905,1.266985,0.468437,0.454265,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.475706,1.033018,-0.971420,0.621769,0.683373,0.753742,0.087333,-0.429313,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,0,1,1,6
602564,13304,1,0.0,191.0,1.022000,-0.401498,0.500221,0.057592,-0.275272,1.023658,0.874287,-0.451520,-0.063366,-0.629891,-0.657284,-1.056912,0.328327,0.054803,1.115859,1.242303,1.044368,0.703425,-0.894861,-0.561509,0.634098,0.468437,0.135578,0.024764,0.223144,-0.248615,-0.186168,-0.391198,0.047367,0.803185,-0.573219,0.344320,-0.630777,0.224456,0.072557,0.753742,-0.710447,-0.413381,-0.136872,-0.483745,0.499599,-0.350631,0.950335,4,1,1,1,6
602565,13304,1,0.0,

## Test and validation set

## Training

## Evaluate and visualize results

## Result and discussion